# Introduction to Keys and Signatures 

This is a short introduction to digital signatures by [Francesco Roda](mailto:francesco.roda@citi.com).  
Start from the top cell of the workbook and run each cell by pressing the play button at the top of the window ▶️

**The code**    
Below, we define the functions to generate a SECP256K1 key pair, sign a message with the private key and verify the digital signature with the public key.  
The code below uses a standard library, ecdsa, which contains the definitions for the SECP256K1 cryptographic functions.  
In the working below we let the system generate a private key at random. In the next session we will explore interesting alternatives. 

In [10]:
import ecdsa 
import binascii

def generate_keys():
    kpr = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
    private_key = kpr.to_string().hex()
    kpu = kpr.get_verifying_key() 
    public_key = kpu.to_string().hex()
    return public_key, private_key #, kpr
def derive_public_key(private_key_asc):
    #kpr_bytes =codecs.decode(private_key_asc, 'hex') #back to bytes
    kpr_bytes = binascii.unhexlify(private_key_asc)
    kpu = ecdsa.SigningKey.from_string(kpr_bytes, curve=ecdsa.SECP256k1).verifying_key
    #kpu_hex = '04' + codecs.encode(kpu.to_string(), 'hex').decode("utf-8")  #back to string
    kpu_hex = '04' + binascii.b2a_hex(kpu.to_string()).decode("utf-8")
    return kpu_hex
def sign_message(message, private_key): 
    bmessage = message.encode()
    sk = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)
    signature = sk.sign(bmessage)
    return signature, message
def verify_signature(public_key, signature, message): 
    vk = ecdsa.VerifyingKey.from_string(bytes.fromhex(public_key), curve=ecdsa.SECP256k1)
    try: 
      vk.verify(signature, message.encode())
      return True 
    except:
      return False
  

print("Functions defined, you can now use them. Remember to enclose strings of characters like messages and keys within quotes  - for example \'string of characters\'")

Functions defined, you can now use them. Remember to enclose strings of characters like messages and keys within quotes  - for example 'string of characters'


**Create a key pair**  
We use the generate_key() function to randomly generate a private key and derive the corresponding public key.  
The author (sender of the message) will use the private key to digitally sign a message. Readers will use the public key of the author to verify the integrity of the message published.  
The author publishes his public key, the message and the digital signature. Of course the author does not publish the private key.  

In [11]:
#public_key, private_key, private_key_u = generate_keys()
public_key, private_key = generate_keys()
print("Public key:", public_key)
print("Private key:", private_key)


Public key: 22e0e82df2ba1d1847e9e2ad5911e044576fa27b941dd49b2b98092a9351b5c925f23fb3caf560ee8b98a226fa89ce38477a7c5ed9e4bcd7d5099761629f5bc1
Private key: 0520f14bc55a02732af0620608ac5df7b46bf39edf95fb3b9e013be50e870543


**The sender uses his private key to sign a message**  
We use the sign_message() function to sign a message with the private key.

In [56]:
signature, message = sign_message("I owe John 21 pounds, Sincerely Francesco Roda", private_key)
print("Message: ", message)
print("Signature: ",signature.hex())

Message:  I owe John 21 pounds, Sincerely Francesco Roda
Signature:  07b90887b3230aebcd60f52a0fe8272148675cd9ecfa3260dff5616f005c782ae4c4a7b808a08016c723256772fda99a3fe185c10ceee09debf0eb85c87bd69c


**The recipient uses the sender public key to verify the integrity of the message**  
We use the verify_signature() function to check the signature for a given message and a given private key.

In [54]:
test = verify_signature(public_key, signature, message) #this is what miners do 
print("The digital signature proves the integrity of the message", test)

The digital signature proves the integrity of the message True


**The recipient uses the sender public key to verify the integrity of another message with the same digital signature**  
We use the verify_signature() function to check the signature for a given message and a given private key.

In [55]:
message_changed = "I owe John 19 pounds, Sincerely Francesco Roda"
print("New message: ", message_changed)
test = verify_signature(public_key, signature, message_changed)
print("The digital signature proves the integrity of the message", test) 

New message:  I owe John 19 pounds, Sincerely Francesco Roda
The digital signature proves the integrity of the message False


**"Sending" messages**  
Digitally signed messages can be send via email to the intended recipient but they do not guarantee privacy: if the message is intercepted, its content is clear for everyone to read.  
Public key cryptography can be used to enforce privacy of cimmunications.  
The recipe is as follows:  
> The recipient publishes her public key  
> The sender encrypts the message with her public key of the recipient  
> The encrypted message is published: there is no need to send it as no one but the recipient can decrypt it       
> The recipient decrypts the message with her private key to access its content   
>
💡 Public key cryptography is also called asymettric cryptography: the asymmetry refers to the role of the public key (encypt messages) and the need for the private key to decrypt them.
💡 More formally:   
> a single key is required to encypt a message (the public key of the recipient)  
> both keys are required to decrypt a message  
> the public key can be derived from the private key but **not vice versa**   
>
The code below defines functions that use the SECP256K1 algorithm to encrypt and decrypt messages.   

In [1]:
from ecies.utils import generate_eth_key
from ecies.utils import generate_eth_key
from ecies import encrypt, decrypt
def encrypt_message(public_key, message):
    pubhex =  public_key
    msghex = message.encode()
    return encrypt(pubhex, msghex)
def decrypt_message(private_key, message):
    prvhex = private_key
    msghex = message
    return decrypt(prvhex, message).decode()

**Group Exercise**    
Work in pairs to complete the following tasks:  
> 1) Exchange a signed message and verify its integrity
> 2) Exchange and encrypted message
> 3) Exchange an encrypted and signed message  
>
💡 To complete these task each player would need to create a key pair and publish her public key  
*Use the functions defined in this workbook as a reminder they are listed below*  
        generate_keys()  
        derive_public_key(private_key_asc), *returns public_key, private_key, kpr*  
        sign_message(message, private_key)  
        verify_signature(public_key, signature, message)  
        encrypt_message(public_key, message)  
        decrypt_message(private_key, message)  
        
        

        

In [7]:
generate_keys()

Private key:  42414931c4e2ab3a77405ecef3ad0b059b959e75951aca27911ae3745db9d0f5
Public key:  6f629d22c7a48c75913ccd59b41a6ff8ca9829f54923208d221c257368e50b932286d0532131e45b248b802c7712d30769d199fbc119a9133560c55737c76e5f


('6f629d22c7a48c75913ccd59b41a6ff8ca9829f54923208d221c257368e50b932286d0532131e45b248b802c7712d30769d199fbc119a9133560c55737c76e5f',
 '42414931c4e2ab3a77405ecef3ad0b059b959e75951aca27911ae3745db9d0f5',
 <ecdsa.keys.SigningKey at 0x7f27f892fe20>)